In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_datareader import data, wb
import datetime
from datetime import date, timedelta
import math
from sklearn.metrics import mean_squared_error 
%matplotlib inline

In [2]:
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2020, 1, 1)

In [3]:
bank="C"
stocks = data.DataReader(bank, 'yahoo', start, end)
colname='Open'

In [4]:
df=stocks.reset_index()[colname]

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [6]:
train_size=int(len(df)*0.70)
test_size=len(df)-train_size
train_data=df[0:train_size]
test_data=df[train_size:]

In [7]:
def make_dataset(data,timestep):
    X=[]
    y=[]
    n=len(data)
    for i in range(0,n-timestep-1):
        X.append(data[i:i+timestep])
        y.append(data[i+timestep])
    return np.array(X),np.array(y)

In [8]:
def errorpercentage(y_test,pred):
    mape=np.mean(np.abs(y_test-pred)/y_test*100)
    return mape

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [10]:
maxstep=26
minstep=25
final_timestep=minstep
r=100
ep=100
results=np.zeros(maxstep)
for timestep in range(minstep,maxstep):
    X_train,y_train=make_dataset(train_data,timestep)
    X_test,y_test=make_dataset(test_data,timestep)
    X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)
    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(timestep,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=ep,batch_size=40,verbose=0)
    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)
    train_predict=scaler.inverse_transform(train_predict)
    test_predict=scaler.inverse_transform(test_predict)
    results[timestep]=mean_squared_error (scaler.inverse_transform(y_test),test_predict)

In [11]:
def findmin(arr):
    ans=100
    idx=0
    c=0
    for i in arr:
        if(i<ans and i!=0):
            ans=i
            idx=c
        c=c+1
    return (ans,idx)

In [12]:
results=results.reshape(maxstep,1)

In [13]:
results[minstep:]


array([[2.45112099]])

In [14]:
minimum_error,selectedstep=findmin(results)

In [15]:
print('minimum error obtained will be ',minimum_error[0],'and the optimum timestep is',selectedstep)

minimum error obtained will be  2.451120991729202 and the optimum timestep is 25
